In [1]:
import pandas as pd
import scipy
import numpy as np
from numpy import array

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from mape import mean_absolute_percentage_error

from IPython.display import clear_output
from pandas import ExcelWriter

import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.models import model_from_json
from keras.layers import Dropout

from data_reshape import univariate_reshape

Using TensorFlow backend.


In [2]:
path = r'power_dataset.xlsx'
data = pd.read_excel (path)
target = data.power.iloc[-24*7:]
target = target.reset_index(drop=True)
dataset = data.power.iloc[:-24*7]
dataset = pd.DataFrame(dataset)

In [3]:
# Normalize Dataset )
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [6]:
# load LSTM MODEL
json_file = open('features parameter Revisi LSTM model number 0.json', 'r')
# json_file = open('LSTM5.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_lstm = model_from_json(loaded_model_json)

# load weights into new model
model_lstm.load_weights("features parameter Revisi LSTM model number 0.h5")

In [7]:
# load RNN MODEL
json_file = open('features parameter Revisi RNN model number 0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_rnn = model_from_json(loaded_model_json)

# load weights into new model
model_rnn.load_weights("features parameter Revisi RNN model number 0.h5")

In [ ]:
#LSTM 
in_steps_lstm = 1
in_features_lstm = 336
out_steps_lstm = 168
train_x,train_y,test_sample = univariate_reshape(dataset,in_steps_lstm,in_features_lstm,out_steps_lstm,target)

predict = model_lstm.predict(test_sample)
predict_lstm = scaler.inverse_transform(predict)

In [ ]:
#RNN
in_steps_rnn = 1
in_features_rnn = 336
out_steps_rnn = 168
train_x,train_y,test_sample = univariate_reshape(dataset,in_steps_rnn,in_features_rnn,out_steps_rnn,target)

predict = model_rnn.predict(test_sample)
predict_rnn = scaler.inverse_transform(predict)

In [ ]:
#ANN
in_steps_ann = 1
in_features_ann = 168
out_steps_ann = 168
train_x,train_y,test_sample = univariate_reshape(dataset,in_steps_ann,in_features_ann,out_steps_ann,target)
test_sample = test_sample.reshape(test_sample.shape[0],test_sample.shape[2])

predict = model_ann.predict(test_sample)
predict_ann = scaler.inverse_transform(predict)

In [113]:
j = 0

in_steps = 1
in_features = 168
out_steps = 168
    
# load ANN MODEL
json_file = open('hidden layer parameter Revisi ANN model number 0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_ann = model_from_json(loaded_model_json)

# load weights into new model
model_ann.load_weights("hidden layer parameter Revisi ANN model number 0.h5")
    
simpan_aktual = []
simpan_prediksi = []
simpan_rmse = []
simpan_mape = []
    
for i2 in range(30):
        
    print("Testing Model {}/{}\n".format(j,30))
    
    batas_bawah_target = 30 - i2
    batas_atas_target = batas_bawah_target-1
    if batas_atas_target == 0:
        target = data.power.iloc[-batas_bawah_target*24*7:]
    else :
        target = data.power.iloc[-batas_bawah_target*24*7:-batas_atas_target*24*7]
        
    target = target.reset_index(drop=True)
    dataset_trg = data.power.iloc[:-batas_bawah_target*24*7]
    dataset_trg = pd.DataFrame(dataset_trg)
        
    # Normalize Dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset_trg = scaler.fit_transform(dataset_trg)
    _ , _ ,test_sample = univariate_reshape(dataset_trg,in_steps,in_features,out_steps,target)
    test_sample = test_sample.reshape(test_sample.shape[0],test_sample.shape[2])
        
    predict = model_ann.predict(test_sample)
    predict = scaler.inverse_transform(predict)
    
    RMSE = np.sqrt(mean_squared_error(target,predict[0]))
    MAPE = mean_absolute_percentage_error(target,predict[0])
    
    simpan_rmse.append(RMSE)
    simpan_mape.append(MAPE)
        
    simpan_aktual.append(target.values)
    simpan_prediksi.append(predict[0])
        
    if i2 == 29 :
        sp = np.array(simpan_prediksi)
        sp = sp.flatten()
        sa = np.array(simpan_aktual)
        sa = sa.flatten()

        simpan_total = pd.DataFrame(columns = ['Aktual','Prediksi'])
        simpan_total.Aktual = sa
        simpan_total.Prediksi = sp
            
    j = j+1
    clear_output(wait=False)
    
simpan_rmse_ANN = simpan_rmse
simpan_mape_ANN = simpan_mape

In [114]:
simpan_total

,Aktual,Prediksi
0,250.794584,238.359634
1,240.518250,225.909454
2,227.525265,212.946289
3,223.913432,214.687607
4,219.601690,209.780869
...,...,...
5035,297.121263,298.706451
5036,312.122241,295.998383
5037,308.669385,285.277283
5038,287.074941,265.078827


In [110]:
j = 0

in_steps = 1
in_features = 336
out_steps = 168
    
# load ANN MODEL
json_file = open('features parameter Revisi RNN model number 0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_rnn = model_from_json(loaded_model_json)

# load weights into new model
model_rnn.load_weights("features parameter Revisi RNN model number 0.h5")
    
simpan_aktual = []
simpan_prediksi = []
simpan_rmse = []
simpan_mape = []
    
for i2 in range(30):
        
    print("Testing Model {}/{}\n".format(j,30))
        
    batas_bawah_target = 30 - i2
    batas_atas_target = batas_bawah_target-1
    if batas_atas_target == 0:
        target = data.power.iloc[-batas_bawah_target*24*7:]
    else :
        target = data.power.iloc[-batas_bawah_target*24*7:-batas_atas_target*24*7]
    
    target = target.reset_index(drop=True)
    dataset_trg = data.power.iloc[:-batas_bawah_target*24*7]
    dataset_trg = pd.DataFrame(dataset_trg)
        
    # Normalize Dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset_trg = scaler.fit_transform(dataset_trg)
    _ , _ ,test_sample = univariate_reshape(dataset_trg,in_steps,in_features,out_steps,target)
        
    predict = model_rnn.predict(test_sample)
    predict = scaler.inverse_transform(predict)
    
    RMSE = np.sqrt(mean_squared_error(target,predict[0]))
    MAPE = mean_absolute_percentage_error(target,predict[0])
    
    simpan_rmse.append(RMSE)
    simpan_mape.append(MAPE)
        
    simpan_aktual.append(target.values)
    simpan_prediksi.append(predict[0])
        
    if i2 == 29 :
        sp = np.array(simpan_prediksi)
        sp = sp.flatten()
        sa = np.array(simpan_aktual)
        sa = sa.flatten()

        simpan_total = pd.DataFrame(columns = ['Aktual','Prediksi'])
        simpan_total.Aktual = sa
        simpan_total.Prediksi = sp
            
    j = j+1
    clear_output(wait=False)
    
simpan_rmse_RNN = simpan_rmse
simpan_mape_RNN = simpan_mape

In [111]:
simpan_total

,Aktual,Prediksi
0,250.794584,257.751373
1,240.518250,244.414368
2,227.525265,224.258881
3,223.913432,221.043167
4,219.601690,220.394699
...,...,...
5035,297.121263,309.651703
5036,312.122241,306.842468
5037,308.669385,309.859070
5038,287.074941,287.463745


In [112]:
data.iloc[-1]

Unnamed: 0    22679.000000
power           262.454059
Name: 22679, dtype: float64

In [73]:
135-75

60

In [65]:
data.power.iloc[-168*16]

227.2399324178696

In [ ]:
# Koreksi

#LSTM ANN
from scipy.stats import wilcoxon
stat, p = scipy.stats.ranksums(dfhasil.lstm, dfhasil.ann)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Sample is not significantly different (accept H0)')
else:
    print('Sample is significantly different (reject H0)')
    
print("")

#LSTM RNN
[stat,pval] = scipy.stats.ttest_ind(dfhasil.lstm,dfhasil.rnn)
alpha = 0.05;
print('T stat = {}'.format(stat))
print('P Value = {}'.format(pval))
if pval < alpha:
    print('Sample is significantly different (reject H0)')
else:
    print('Sample is not significantly different (accept H0)')

print("")

#LSTM ARIMA
[stat,pval] = scipy.stats.ttest_ind(dfhasil.lstm,dfhasil.arima)
alpha = 0.05;
print('T stat = {}'.format(stat))
print('P Value = {}'.format(pval))
if pval < alpha:
    print('Sample is significantly different (reject H0)')
else:
    print('Sample is not significantly different (accept H0)')